In [450]:
import numpy as np
import math as m
import itertools
import scipy as sp

def binary_decode(bytearray: list[int]):
    bytearray = np.flip(bytearray)
    return sum(bytearray * (2 ** np.array(list(enumerate(bytearray.flat)))[:,0]))

def binary_encode(ingest: list[int], meta: list[int]):
    return np.floor_divide(ingest, (2 ** np.array(list(enumerate(meta)))[:,0])) % 2

def func(i):
    return m.tanh(i)

def float_func(i, precision):
    val = i * (2**(precision-1))
    bin_val = binary_encode(val,range(0,(precision)))
    return np.flip(bin_val)

def float_decomp(i, precision):
    exponent = np.arange(0,precision)
    div = np.array([2]*(precision))
    div = div ** exponent
    div = np.array([1]*(precision)) / div
    return np.sum(div*i)

bin_inputs = np.array(list(itertools.product([0, 1], repeat=8)))

bin_inputs[:,0] = bin_inputs[:,0] * 0

inputs = np.apply_along_axis(float_decomp, 1, bin_inputs,precision=8)

index_inputs =  np.array(list(enumerate(inputs.tolist())))[:,0].astype(int)

outputs = np.array([func(i) for i in inputs])

bin_outputs = np.array([float_func(o,8) for o in outputs])

index_outputs = np.array([binary_decode(bo) for bo in bin_outputs]).astype(int) + 1

wildcards = np.vstack([index_inputs,index_outputs]).T
halves = np.array(np.unique(index_outputs,return_counts=True, axis=0))
mixed = np.ones(index_outputs.shape)
mixed[halves[0,:]] = halves[1,:]
superimposed_values = mixed[wildcards[:,1]]

density = sp.sparse.csr_matrix(((1/np.sqrt(superimposed_values)),(index_inputs,index_outputs)),(index_inputs.size,index_outputs.size)).toarray()

input_vectors_matrix = np.diag(np.ones(index_inputs.shape))

output_vectors_matrix = sp.sparse.csr_matrix((np.ones(index_outputs.shape),(index_inputs,index_outputs)),(index_inputs.size,index_outputs.size)).toarray()

unitary = np.linalg.qr(output_vectors_matrix)[0]

print(unitary)

[[ 1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -0.57735027  0.         ...  0.          0.
   0.        ]
 [ 0.         -0.         -0.70710678 ...  0.          0.
   0.        ]
 ...
 [ 0.         -0.         -0.         ...  0.75       -0.25
   0.        ]
 [ 0.         -0.         -0.         ... -0.25        0.75
   0.        ]
 [ 0.         -0.         -0.         ...  0.          0.
   0.5       ]]


In [451]:
def testing_unitary(dec_input, precision):

    input = float_func(dec_input, precision)

    input_basis_state = np.zeros(2**precision)

    input_basis_state[int(binary_decode(input))] = 1

    output_basis_state = input_basis_state @ unitary

    #output_basis_state = input_basis_state @ density

    prob_indexes = output_basis_state.nonzero()[0]

    probs = output_basis_state[prob_indexes]

    max_prob = np.argmax(output_basis_state)

    print(probs)

    print(prob_indexes)

    bin_outputs = [binary_encode(output, range(0,(precision))) for output in prob_indexes]

    print(binary_decode(float_func(func(dec_input),precision)))

testing_unitary(0.52,8)

testing_unitary(0.42,8)

testing_unitary(0.32,8)

testing_unitary(0.12,8)

[-0.70710678  0.03608439  0.03608439  0.03608439  0.04419417  0.04419417
  0.04419417  0.04419417  0.04419417  0.04419417 -0.5        -0.25
 -0.25        0.1767767   0.1767767  -0.08838835 -0.08838835 -0.08838835
 -0.08838835  0.03608439  0.03608439  0.03608439  0.04419417  0.04419417
  0.04419417  0.04419417  0.04419417  0.04419417]
[ 61 118 119 120 121 122 123 124 125 126 194 200 201 209 210 222 223 224
 225 246 247 248 249 250 251 252 253 254]
61.0
[-0.70710678 -0.02209709 -0.02209709 -0.00552427 -0.00552427 -0.00552427
 -0.00552427 -0.0078125  -0.00552427 -0.00552427 -0.01104854 -0.01104854
 -0.0078125  -0.0078125  -0.0078125  -0.0078125  -0.01104854 -0.01104854
 -0.5        -0.35355339  0.1767767   0.1767767  -0.125      -0.08838835
 -0.08838835  0.08838835  0.04419417  0.04419417  0.0625     -0.0625
 -0.02209709 -0.02209709 -0.03125    -0.03125    -0.03125     0.04419417
  0.01104854  0.01104854  0.01104854  0.01104854  0.02209709  0.015625
  0.015625    0.02209709 -0.02209709 -0

In [452]:
precision = 8

num_tests = 15

test_inputs = np.random.rand(num_tests)

input_basis_states = np.array([np.zeros(2**precision) for test_input in test_inputs])

index_test_inputs = np.array([binary_decode(float_func(test_input,precision)) for test_input in test_inputs]).astype(int)

print(input_basis_states[np.arange(0,index_test_inputs.size),index_test_inputs] )

input_basis_states[np.arange(0,index_test_inputs.size),index_test_inputs] = np.ones(index_test_inputs.shape)

print(input_basis_states[np.arange(0,index_test_inputs.size),index_test_inputs] )

predictions = np.array([(predict @ unitary).nonzero()[0][0] for predict in input_basis_states])

print(predictions)

actuals = np.array([int(binary_decode(float_func(func(real),precision))) for real in (test_inputs)])

print(actuals)

predictions - actuals

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 9 68 76 21 64 76  7 12 67 50 54 52 37 18 22]
[ 9 68 75 21 63 75  7 12 67 50 54 52 36 18 22]


array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [453]:
precision = 8

num_tests = 15

test_inputs = np.random.rand(num_tests)

input_basis_states = np.array([np.zeros(2**precision) for test_input in test_inputs])

index_test_inputs = np.array([binary_decode(float_func(test_input,precision)) for test_input in test_inputs]).astype(int)

print(input_basis_states[np.arange(0,index_test_inputs.size),index_test_inputs] )

input_basis_states[np.arange(0,index_test_inputs.size),index_test_inputs] = np.ones(index_test_inputs.shape)

print(input_basis_states[np.arange(0,index_test_inputs.size),index_test_inputs] )

predictions = np.array([float_decomp(np.flip(binary_encode((predict @ unitary).nonzero()[0][0],np.arange(0,precision))),precision) for predict in input_basis_states])

print(predictions)

actuals = np.array([func(real) for real in (test_inputs)])

print(actuals)

np.average(predictions - actuals)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0.6484375 0.671875  0.5234375 0.34375   0.1640625 0.0390625 0.421875
 0.5625    0.578125  0.671875  0.6171875 0.3984375 0.7578125 0.4453125
 0.4609375]
[0.64763721 0.673881   0.52515225 0.33783499 0.16591658 0.04520991
 0.422385   0.56168415 0.58099706 0.66985559 0.62026094 0.39820947
 0.7520156  0.44921924 0.45628152]


-0.00012353348871097143